In [1]:
import pandas as pd
import glob
import os
import numpy as np
from time import time
import logging
import gensim
import bz2
import re
from nltk.data import load
import random
import nltk



/Users/Barry/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
train_df = pd.read_csv('../datas/tweet/dataset.csv')
train_df.head()

,Text,sentiment
0,Et quand est arrivée avec sa wifi dans la criq...,0
1,"RT L'histoire des fans de kpop en Europe , Afr...",0
2,Plus le temps pour le moment :(,0
3,je l'aide du mieux que je peux par message mai...,0
4,J'ai rien fait de la journée et je suis fatigu...,0


In [3]:
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC

naive_bayes = NaiveBayesClassifier.train
svm = SklearnClassifier(LinearSVC()).train
maxent = MaxentClassifier.train



In [4]:
def parse_tweets_set(data, label, word_tokenizer=None, sent_tokenizer=None,
                     skip_header=True):
    """
    Parse csv file containing tweets and output data a list of (text, label) tuples.

    :param filename: the input csv filename.
    :param label: the label to be appended to each tweet contained in the csv file.
    :param word_tokenizer: the tokenizer instance that will be used to tokenize
        each sentence into tokens (e.g. WordPunctTokenizer() or BlanklineTokenizer()).
        If no word_tokenizer is specified, tweets will not be tokenized.
    :param sent_tokenizer: the tokenizer that will be used to split each tweet into
        sentences.
    :param skip_header: if True, skip the first line of the csv file (which usually
        contains headers).

    :return: a list of (text, label) tuples.
    """
    tweets = []
    # If we use Python3.x we can proceed using the 'rt' flag
    if sys.version_info[0] == 3:
            i = 0
            for text in data.Text:
                # text = text[1]
                i += 1
                sys.stdout.write('Loaded {0} tweets\r'.format(i))
                # Apply sentence and word tokenizer to text
                if word_tokenizer:
                    tweet = [w for w in word_tokenizer.tokenize(text)]
                else:
                    tweet = text
                tweets.append((tweet, label))
    print("Loaded {0} tweets".format(i))
    return tweets

In [5]:
from nltk.tokenize import TweetTokenizer
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import twitter_samples, stopwords
tokenizer = TweetTokenizer(preserve_case=False)
import sys

pos_docs = parse_tweets_set(train_df[train_df['sentiment']==1], label='pos', word_tokenizer=tokenizer)
neg_docs = parse_tweets_set(train_df[train_df['sentiment']==0], label='neg', word_tokenizer=tokenizer)

/Users/Barry/anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Loaded 7027 tweets
Loaded 9745 tweets


In [6]:
def split_train_test(all_instances, n=None):
    """
    Randomly split `n` instances of the dataset into train and test sets.

    :param all_instances: a list of instances (e.g. documents) that will be split.
    :param n: the number of instances to consider (in case we want to use only a
        subset).
    :return: two lists of instances. Train set is 8/10 of the total and test set
        is 2/10 of the total.
    """
    random.seed(12345)
    random.shuffle(all_instances)
    if not n or n > len(all_instances):
        n = len(all_instances)
    train_set = all_instances[:int(.8*n)]
    test_set = all_instances[int(.8*n):n]

    return train_set, test_set

In [7]:
def extract_unigram_feats(document, unigrams, handle_negation=False):
    """
    Populate a dictionary of unigram features, reflecting the presence/absence in
    the document of each of the tokens in `unigrams`.

    :param document: a list of words/tokens.
    :param unigrams: a list of words/tokens whose presence/absence has to be
        checked in `document`.
    :param handle_negation: if `handle_negation == True` apply `mark_negation`
        method to `document` before checking for unigram presence/absence.
    :return: a dictionary of unigram features {unigram : boolean}.

    >>> words = ['ice', 'police', 'riot']
    >>> document = 'ice is melting due to global warming'.split()
    >>> sorted(extract_unigram_feats(document, words).items())
    [('contains(ice)', True), ('contains(police)', False), ('contains(riot)', False)]
    """
    features = {}
    if handle_negation:
        document = mark_negation(document)
    for word in unigrams:
        features['contains({0})'.format(word)] = word in set(document)
    return features

def extract_bigram_feats(document, bigrams):
    """
    Populate a dictionary of bigram features, reflecting the presence/absence in
    the document of each of the tokens in `bigrams`. This extractor function only
    considers contiguous bigrams obtained by `nltk.bigrams`.

    :param document: a list of words/tokens.
    :param unigrams: a list of bigrams whose presence/absence has to be
        checked in `document`.
    :return: a dictionary of bigram features {bigram : boolean}.

    >>> bigrams = [('global', 'warming'), ('police', 'prevented'), ('love', 'you')]
    >>> document = 'ice is melting due to global warming'.split()
    >>> sorted(extract_bigram_feats(document, bigrams).items())
    [('contains(global - warming)', True), ('contains(love - you)', False),
    ('contains(police - prevented)', False)]
    """
    features = {}
    for bigr in bigrams:
        features['contains({0} - {1})'.format(bigr[0], bigr[1])] = bigr in nltk.bigrams(document)
    return features

In [81]:
trainer = naive_bayes
train_pos_docs, test_pos_docs = split_train_test(pos_docs)
train_neg_docs, test_neg_docs = split_train_test(neg_docs)

training_tweets = train_pos_docs+train_neg_docs
testing_tweets = test_pos_docs+test_neg_docs

sentim_analyzer = SentimentAnalyzer()
# stopwords = stopwords.words('english')
# all_words = [word for word in sentim_analyzer.all_words(training_tweets) if word.lower() not in stopwords]
all_words = [word for word in sentim_analyzer.all_words(training_tweets)]

# Add simple unigram word features
unigram_feats = sentim_analyzer.unigram_word_feats(all_words, top_n=1000)
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

# Add bigram collocation features
bigram_collocs_feats = sentim_analyzer.bigram_collocation_feats([tweet[0] for tweet in training_tweets],
    top_n=100, min_freq=12)
sentim_analyzer.add_feat_extractor(extract_bigram_feats, bigrams=bigram_collocs_feats)

training_set = sentim_analyzer.apply_features(training_tweets)
test_set = sentim_analyzer.apply_features(testing_tweets)
classifier = sentim_analyzer.train(trainer, training_set)
# classifier = sentim_analyzer.train(trainer, training_set, max_iter=4)
try:
    classifier.show_most_informative_features()
except AttributeError:
    print('Your classifier does not provide a show_most_informative_features() method.')
results = sentim_analyzer.evaluate(test_set)


Training classifier
Most Informative Features
            contains(:() = True              neg : pos    =    475.0 : 1.0
           contains(:-() = True              neg : pos    =    180.0 : 1.0
            contains(:)) = True              pos : neg    =    144.2 : 1.0
           contains(:-)) = True              pos : neg    =    126.6 : 1.0
            contains(:D) = True              pos : neg    =     65.8 : 1.0
        contains(triste) = True              neg : pos    =     48.0 : 1.0
            contains(:P) = True              pos : neg    =     43.4 : 1.0
         contains(bravo) = True              pos : neg    =     43.0 : 1.0
            contains(:p) = True              pos : neg    =     35.7 : 1.0
contains(belle - journée) = True              pos : neg    =     33.7 : 1.0
<class 'nltk.util.LazyMap'>
Evaluating NaiveBayesClassifier results...


In [50]:
results

{'Accuracy': 0.9663189269746647,
 'F-measure [neg]': 0.9706569722150092,
 'F-measure [pos]': 0.9604756908009795,
 'Precision [neg]': 0.9826498422712934,
 'Precision [pos]': 0.9449415003441156,
 'Recall [neg]': 0.9589533093894305,
 'Recall [pos]': 0.9765291607396871}

In [159]:
def parse_new_set(data,word_tokenizer=None, sent_tokenizer=None,
                     skip_header=True):
    """
    Parse csv file containing tweets and output data a list of (text, label) tuples.

    :param filename: the input csv filename.
    :param label: the label to be appended to each tweet contained in the csv file.
    :param word_tokenizer: the tokenizer instance that will be used to tokenize
        each sentence into tokens (e.g. WordPunctTokenizer() or BlanklineTokenizer()).
        If no word_tokenizer is specified, tweets will not be tokenized.
    :param sent_tokenizer: the tokenizer that will be used to split each tweet into
        sentences.
    :param skip_header: if True, skip the first line of the csv file (which usually
        contains headers).

    :return: a list of (text, label) tuples.
    """
    tweets = []
    # If we use Python3.x we can proceed using the 'rt' flag
    if sys.version_info[0] == 3:
            i = 0
            for text in data:
                # text = text[1]
                i += 1
                sys.stdout.write('Loaded {0} tweets\r'.format(i))
                # Apply sentence and word tokenizer to text
                if word_tokenizer:
                    tweet = [w for w in word_tokenizer.tokenize(text)]
                else:
                    tweet = text
                tweets.append((tweet, 'neutral'))
    print("Loaded {0} tweets".format(i))
    return tweets

In [166]:
df = pd.read_csv('../datas/scrap/voting/voting_with_topics_unique.csv')

feature = df.text
feature = parse_new_set(feature, word_tokenizer=tokenizer)
new_set = sentim_analyzer.apply_features(feature)

Loaded 3470 tweets


In [167]:
results = classifier.classify_many([fs for (fs, l) in new_set])

In [177]:
results

['pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',


In [170]:
for i in range(len(feature)) :
    print(str(feature[i]) +'SENTIMENT :'+str(results[i]))

(['arrêté', 'fédéral', 'concernant', 'la', 'contribution', 'de', 'la', 'suisse', 'en', 'faveur', 'de', 'la', 'bulgarie', 'et', 'de', 'la', 'roumanie', 'au', 'titre', 'de', 'la', 'réduction', 'des', 'disparités', 'économiques', 'et', 'sociales', 'dans', "l'union", 'européenne', 'élargie', 'réduction', 'des', 'disparités', 'économiques', 'et', 'sociales', 'dans', "l'ue", '.', 'contribution', 'de', 'la', 'suisse', 'en', 'faveur', 'de', 'la', 'roumanie', 'et', 'de', 'la', 'bulgarie'], 'neutral')SENTIMENT :pos
(['renforcement', 'du', 'traité', 'sur', 'la', 'non-prolifération', 'des', 'armes', 'nucléaires'], 'neutral')SENTIMENT :pos
(['une', 'zone', 'exempte', "d'armes", 'nucléaires', 'au', 'coeur', 'de', "l'europe"], 'neutral')SENTIMENT :pos
(['boycott', 'de', 'la', 'liste', 'des', 'terroristes', 'établie', 'par', "l'onu"], 'neutral')SENTIMENT :pos
(['ratification', 'du', 'protocole', 'de', "l'onu", 'sur', 'les', 'armes', 'à', 'feu', 'et', 'mise', 'en', 'oeuvre', 'de', "l'instrument", 'thal